In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_10752\1140549349.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer
scorer = make_scorer(lambda a, b: spearmanr(a, b), greater_is_better=True)

In [3]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select ticker, date, agr, bm, idiovol, mom12m, roeq, ret
    from data
    where date in ('2021-12', '2022-01')
    """, 
    conn
)
features = ["agr", "bm", "idiovol", "mom12m", "roeq"]
df = df.dropna()

In [4]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution="normal")
def qtxs(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features+["ret"]] = df.groupby(
  "date", 
  group_keys=False
)[features+["ret"]].apply(qtxs)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

Xtrain = df[df.date=='2021-12'][features]
ytrain = df[df.date=='2021-12']["ret"]

param_grid = {
  "random_state": [0],
  "max_depth": [3, 4], 
  "learning_rate": [0.05, 0.1]
}

cv = GridSearchCV(
  model,
  param_grid=param_grid,
)

cv.fit(Xtrain, ytrain)
cv.best_params_

{'learning_rate': 0.05, 'max_depth': 3, 'random_state': 0}

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor()
)

param_grid = {
  "base_estimator__random_state": [0],
  "base_estimator__max_depth": [3, 4],
  "learning_rate": [0.1, 0.2]
}

cv = GridSearchCV(
  model,
  param_grid=param_grid
)
cv.fit(Xtrain, ytrain)
cv.best_params_

{'base_estimator__max_depth': 4,
 'base_estimator__random_state': 0,
 'learning_rate': 0.1}

In [7]:
import pandas as pd
pd.DataFrame(cv.cv_results_)

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.213110      0.052232         0.007173        0.003810   
1       0.206245      0.030042         0.005499        0.003253   
2       0.256247      0.050858         0.001631        0.003263   
3       0.251495      0.040703         0.001492        0.002984   

  param_base_estimator__max_depth param_base_estimator__random_state  \
0                               3                                  0   
1                               3                                  0   
2                               4                                  0   
3                               4                                  0   

  param_learning_rate                                             params  \
0                 0.1  {'base_estimator__max_depth': 3, 'base_estimat...   
1                 0.2  {'base_estimator__max_depth': 3, 'base_estimat...   
2                 0.1  {'base_estimator__max_depth': 4, 'base_estimat...   
3                 0.2  {'base_estimator__max_depth': 4, 'base_estimat...   

   split0_test_score  split1_test_score  split2_test_score  split3_test_score  \
0           0.165886           0.173055           0.112494           0.075357   
1           0.124916           0.128976           0.105888           0.059765   
2           0.186847           0.191459           0.113346           0.071036   
3           0.136733           0.164301           0.114194           0.066368   

   split4_test_score  mean_test_score  std_test_score  rank_test_score  
0           0.120271         0.129413        0.036130                2  
1           0.091070         0.102123        0.025184                4  
2           0.140249         0.140588        0.045404                1  
3           0.096053         0.115530        0.033545                3